In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('data/train.csv', index_col = 'Id')
test = pd.read_csv('data/test.csv', index_col = 'Id')
sample_submission = pd.read_csv('data/sample_submission.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
train_id = train.index
test_id = test.index

data = pd.concat([train, test])

### Feature engineering:
- create dummy vars for Product_Info_2 (keep everything else as numeric) Starting from version 1.5, XGBoost has experimental support for categorical data available for public testing.
- calculate sum of all Medical_Keyword columns
- for each binary keyword-value pair, calculate the mean of the target variable, then for each observation take the mean and the minimum of the keyword-value-mean targets

In [ ]:
#create dummy vars for Product_Info_2 (keep everything else as numeric)
data = pd.get_dummies(data, prefix = 'Product_Info_2', columns = ['Product_Info_2'])

In [ ]:
#calculate sum of all Medical_Keyword columns
Medical_Keyword_names = []
for i in range(1, 49):
    Medical_Keyword_names.append('Medical_Keyword_' + str(i))
    
data['Medical_Keyword_sum'] = data[Medical_Keyword_names].sum(axis = 1)

In [ ]:
# for each binary keyword-value pair, calculate the mean of the target variable,
#then for each observation take the mean and the minimum of the keyword-value-mean targets

for value in range(1,8):
    data['target_'+str(value)] = (data['Response'] > value).astype(int)
    temp = pd.DataFrame()
    for col in Medical_Keyword_names:
        replace_dict = dict(data.groupby(by = col)['target_'+str(value)].mean())
        temp[col+'target_'+str(value)] = data[col].replace(replace_dict)
    data['keyword_target_mean_'+str(value)] = temp.sum(axis = 1)
    data['keyword_target_min_'+str(value)] = temp.min(axis = 1)

In [ ]:
data.head(5)

,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_10,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_15,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_24,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_32,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response,Product_Info_2_A1,Product_Info_2_A2,Product_Info_2_A3,Product_Info_2_A4,Product_Info_2_A5,Product_Info_2_A6,Product_Info_2_A7,Product_Info_2_A8,Product_Info_2_B1,Product_Info_2_B2,Product_Info_2_C1,Product_Info_2_C2,Product_Info_2_C3,Product_Info_2_C4,Product_Info_2_D1,Product_Info_2_D2,Product_Info_2_D3,Product_Info_2_D4,Product_Info_2_E1,Medical_Keyword_sum,target_1,keyword_target_mean_1,keyword_target_min_1,target_2,keyword_target_mean_2,keyword_target_min_2,target_3,keyword_target_mean_3,keyword_target_min_3,target_4,keyword_target_mean_4,keyword_target_min_4,target_5,keyword_target_mean_5,keyword_target_min_5,target_6,keyword_target_mean_6,keyword_target_min_6,target_7,keyword_target_mean_7,keyword_target_min_7
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,1,10,0.076923,2,1,1,0.641791,0.581818,0.148536,0.323008,0.028,12,1,0.0,3,NaN,1,2,6,3,1,2,1,1,1,3,1,0.000667,1,1,2,2,NaN,0.598039,NaN,0.526786,4.0,112,2,1,1,3,2,2,1,NaN,3,2,3,3,240.0,3,3,1,1,2,1,2,3,NaN,1,3,3,1,3,2,3,NaN,1,3,1,2,2,1,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,32.329096,0.671625,1,28.415945,0.588901,1,27.802288,0.576012,1,26.929399,0.557951,1,23.646403,0.489238,1,16.847428,0.347117,1,11.983588,0.245507
5,1,26,0.076923,2,3,1,0.059701,0.600000,0.131799,0.272288,0.000,1,3,0.0,2,0.0018,1,2,6,3,1,2,1,2,1,3,1,0.000133,1,3,2,2,0.188406,NaN,0.084507,NaN,5.0,412,2,1,1,3,2,2,1,NaN,3,2,3,3,0.0,1,3,1,1,2,1,2,3,NaN,1,3,3,1,3,2,3,NaN,3,1,1,2,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,1,0,0,0,0,

### Null Value

In [ ]:
#checking percentage of missing values in a column
missing_val_count_by_column = data.isnull().sum()/len(data)
column_w_missing_val = list(data.columns[missing_val_count_by_column > 0])
column_w_missing_val.remove('Response')
print(missing_val_count_by_column[column_w_missing_val].sort_values(ascending=False))

Medical_History_10     0.990423
Medical_History_32     0.981578
Medical_History_24     0.937066
Medical_History_15     0.751270
Family_Hist_5          0.700414
Family_Hist_3          0.572423
Family_Hist_2          0.486898
Insurance_History_5    0.423281
Family_Hist_4          0.326751
Employment_Info_6      0.184987
Medical_History_1      0.149862
Employment_Info_4      0.112653
Employment_Info_1      0.000278
dtype: float64


In [ ]:
data[column_w_missing_val].describe()

,Employment_Info_1,Employment_Info_4,Employment_Info_6,Insurance_History_5,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_10,Medical_History_15,Medical_History_24,Medical_History_32
count,79124.000000,70230.000000,64505.000000,45645.000000,40610.000000,33841.000000,53285.000000,23711.000000,67285.000000,758.000000,19686.000000,4981.000000,1458.000000
mean,0.077930,0.006340,0.363228,0.001710,0.474554,0.498315,0.445338,0.486604,7.928572,143.426121,124.240221,50.449709,11.718107
std,0.082746,0.033368,0.350589,0.006465,0.154993,0.140164,0.163434,0.129180,12.912942,106.499108,98.668380,77.921630,38.257761
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.035000,0.000000,0.060000,0.000400,0.362319,0.411765,0.323944,0.410714,2.000000,9.250000,18.000000,1.000000,0.000000
50%,0.060000,0.000000,0.250000,0.000933,0.463768,0.519608,0.436620,0.508929,4.000000,225.000000,117.000000,8.000000,0.000000
75%,0.100000,0.000000,0.580000,0.002000,0.579710,0.607843,0.563380,0.580357,9.000000,240.000000,240.000000,63.000000,2.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,240.000000,240.000000,240.000000,240.000000,240.000000


In [ ]:
data.fillna(value =-1, inplace = True )

In [ ]:
data.info(verbose=True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79146 entries, 2 to 79129
Data columns (total 167 columns):
 #    Column                 Non-Null Count  Dtype  
---   ------                 --------------  -----  
 0    Product_Info_1         79146 non-null  int64  
 1    Product_Info_3         79146 non-null  int64  
 2    Product_Info_4         79146 non-null  float64
 3    Product_Info_5         79146 non-null  int64  
 4    Product_Info_6         79146 non-null  int64  
 5    Product_Info_7         79146 non-null  int64  
 6    Ins_Age                79146 non-null  float64
 7    Ht                     79146 non-null  float64
 8    Wt                     79146 non-null  float64
 9    BMI                    79146 non-null  float64
 10   Employment_Info_1      79146 non-null  float64
 11   Employment_Info_2      79146 non-null  int64  
 12   Employment_Info_3      79146 non-null  int64  
 13   Employment_Info_4      79146 non-null  float64
 14   Employment_Info_5      79146 non-nul

C:\Users\zhang\Anaconda3\envs\huan\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


### Modeling:



In [ ]:
train, test = data.loc[train_id], data.loc[test_id]

target_col = ['target_' + str(i) for i in range(1,8)]


X_train = train.drop(labels = ['Response'] + target_col , axis= 1)
#X_val = val.drop(labels = ['Response', 'target'], axis= 1)
X_test = test.drop(labels = ['Response'] + target_col , axis= 1)
y_train = train[target_col ]
# y_val = val['target']

In [ ]:
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import GradientBoostingClassifier

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
base_gb = GradientBoostingClassifier(
    n_estimators = 200,
    max_depth = 3, 
    max_features='auto',
    subsample=0.8,
    n_iter_no_change=6,
    validation_fraction=0.2,
    verbose = 1
)

In [ ]:
chain = ClassifierChain(
    base_gb, 
    order = None, # If None, the order will be determined by the order of columns in the label matrix Y
    cv = None, # None, to use true labels when fitting
)

chain.fit(X_train, y_train)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.6570           0.0142            1.08m
         2           0.6441           0.0111            1.06m
         3           0.6375           0.0082            1.06m
         4           0.6297           0.0062            1.06m
         5           0.6236           0.0049            1.05m
         6           0.6152           0.0050            1.04m
         7           0.6167           0.0032            1.04m
         8           0.6079           0.0039            1.03m
         9           0.6055           0.0025            1.02m
        10           0.6021           0.0025            1.02m
        20           0.5816           0.0012           57.72s
        30           0.5665           0.0005           54.50s
        40           0.5550           0.0003           51.25s
        50           0.5542           0.0004           48.02s
        60           0.5474           0.0001           44.87s
       

        80           0.3524           0.0002           34.27s
        90           0.3512          -0.0000           32.12s
       100           0.3438           0.0001           30.06s


ClassifierChain(base_estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                          criterion='friedman_mse',
                                                          init=None,
                                                          learning_rate=0.1,
                                                          loss='deviance',
                                                          max_depth=3,
                                                          max_features='auto',
                                                          max_leaf_nodes=None,
                                                          min_impurity_decrease=0.0,
                                                          min_impurity_split=None,
                                                          min_samples_leaf=1,
                                                          min_samples_split=2,
                                                          min_weight

###  Search for cutoffs
- the aim is to find the boundaries that maximize the kappa score
- boundaries are initialized according to the original Response distribution of the training dataset

In [ ]:
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import fmin_powell

In [ ]:
#search for optimal cutoffs
train_preds = chain.predict_proba(X_train).sum(axis = 1) + 1

def digit(cutoff):
    res = []
    for y in list(train_preds):
        if y < cutoff[0]:
            res.append(1)
        elif y < cutoff[1]:
            res.append(2)
        elif y < cutoff[2]:
            res.append(3)
        elif y < cutoff[3]:
            res.append(4)
        elif y < cutoff[4]:
            res.append(5)
        elif y < cutoff[5]:
            res.append(6)
        elif y < cutoff[6]:
            res.append(7)
        else: res.append(8)
    return res  
    
def train_offset(cutoff):
    res = digit(cutoff)    
    return -cohen_kappa_score(train['Response'], res, weights = 'quadratic') 

cutoff = [1.5,2.9,3.1,4.5,5.5,6.1,7.1]  
offsets = fmin_powell(train_offset, x0 = cutoff, disp = False)

### Final Result

In [ ]:
prob_test = chain.predict_proba(X_test).sum(axis = 1) + 1

res = []
for y in list(prob_test):
    if y < offsets[0]:
        res.append(1)
    elif y < offsets[1]:
        res.append(2)
    elif y < offsets[2]:
        res.append(3)
    elif y < offsets[3]:
        res.append(4)
    elif y < offsets[4]:
        res.append(5)
    elif y < offsets[5]:
        res.append(6)
    elif y < offsets[6]:
        res.append(7)
    else: res.append(8)


sample_submission['Response'] = res

In [ ]:
sample_submission.to_csv('output/sklearn_v3.csv', index=False)